# LMNA: Cardiomyopathy, dilated, 1A 

The data in this workbook were derived from Table 1 of [Fatkin D, et al. (1999) Missense mutations in the rod domain of the lamin A/C gene as causes of dilated cardiomyopathy and conduction-system disease. N Engl J Med 341(23):1715-24. PMID:10580070](https://pubmed.ncbi.nlm.nih.gov/10580070/).


Variants were curated as follows
- Family A: Arg571Ser; C→A (1711): NM_005572.4(LMNA):c.1711C>A (p.Arg571Ser)
- Family B: Asn195Lys; C→G (585): NM_170707.4(LMNA):c.585C>A (p.Asn195Lys) 
- Family C: Leu85Arg; T→G (254): NM_170707.4(LMNA):c.254T>G (p.Leu85Arg) 
- Family D: Arg60Gly; C→G (178): NM_170707.4(LMNA):c.178C>G (p.Arg60Gly)
- Family E: Glu203Gly; A→G (608):  NM_170707.4(LMNA):c.608A>G (p.Glu203Gly)

Note that we will code the following two features as excluded in all individuals based on knowledge external to the above cited publication.

- Excluded: Progeroid facial appearance HP:0005328 (see OMIM)
- Excluded: Lipodystrophy HP:0009125 (see OMIM)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.65


In [2]:
PMID = "PMID:10580070"
title = "Missense mutations in the rod domain of the lamin A/C gene as causes of dilated cardiomyopathy and conduction-system disease"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/Fatkin1999.xlsx")
df.head(2)

,FAMILY,MEMBER,AGE AT CLINICAL PRESENTATION (YR ),Sex,CONDUCTION -SYSTEM DISEASE,ATRIAL FIBRILLATION,DILATED CARDIOMYOPATHY,CAUSE OF DEATH,COMMENTS,CK,Elevated CK,NM_005572.4
0,A,III-1,51,M,"Third-degree atrioventricular block, permanent pacemaker",No,Yes,-,NaN,181 U/liter (<210 U/liter),No,c.1711C>A
1,A,III-5,39,F,NaN,No,No,-,NaN,180 U/liter (<140 U/liter),No,c.1711C>A


In [4]:
def create_id(row):
    return f"{row['FAMILY']}_{row['MEMBER']}"

df["individual_id"] = df.apply(create_id, axis=1)

In [5]:
ageMapper = AgeColumnMapper.by_year(column_name="AGE AT CLINICAL PRESENTATION (YR )")
#ageMapper.preview_column(df["AGE AT CLINICAL PRESENTATION (YR )"])
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")
#sexMapper.preview_column(df["Sex"])

In [6]:
column_mapper_list = list()

In [7]:
# Atrial fibrillation HP:0005110
afMapper = SimpleColumnMapper(column_name='ATRIAL FIBRILLATION',
                              hpo_id="HP:0005110", hpo_label="Atrial fibrillation", observed="Yes", excluded="No")
column_mapper_list.append(afMapper)
afMapper.preview_column(df)

,mapping,count
0,"original value: ""No"" -> HP: Atrial fibrillation (HP:0005110) (excluded)",16
1,"original value: ""Yes"" -> HP: Atrial fibrillation (HP:0005110) (observed)",23


In [8]:
# Dilated cardiomyopathy HP:0001644
dilatedCmMapper = SimpleColumnMapper(column_name='DILATED CARDIOMYOPATHY',
                                     hpo_id="HP:0001644", hpo_label="Dilated cardiomyopathy", observed="Yes", excluded="No")
column_mapper_list.append(dilatedCmMapper)
dilatedCmMapper.preview_column(df)

,mapping,count
0,"original value: ""Yes"" -> HP: Dilated cardiomyopathy (HP:0001644) (observed)",25
1,"original value: ""No"" -> HP: Dilated cardiomyopathy (HP:0001644) (excluded)",14


In [9]:
# Elevated circulating creatine kinase concentration HP:0003236
ckMapper = SimpleColumnMapper(column_name='Elevated CK',
                              hpo_id="HP:0003236", hpo_label="Elevated circulating creatine kinase concentration", observed="Yes", excluded="No")
column_mapper_list.append(ckMapper)
ckMapper.preview_column(df)

,mapping,count
0,"original value: ""No"" -> HP: Elevated circulating creatine kinase concentration (HP:0003236) (excluded)",20
1,"original value: ""nan"" -> HP: Elevated circulating creatine kinase concentration (HP:0003236) (not measured)",18
2,"original value: ""Yes"" -> HP: Elevated circulating creatine kinase concentration (HP:0003236) (observed)",1


In [10]:
#Sudden cardiac death HP:0001645
df['CAUSE OF DEATH'].unique()
suddenMapper = SimpleColumnMapper(column_name='CAUSE OF DEATH',
                                  hpo_id="HP:0001645", hpo_label="Sudden cardiac death", observed="Sudden death", excluded=["No", "-"])
column_mapper_list.append(suddenMapper)
suddenMapper.preview_column(df)

,mapping,count
0,"original value: ""-"" -> HP: Sudden cardiac death (HP:0001645) (excluded)",15
1,"original value: ""Lymphoma"" -> HP: Sudden cardiac death (HP:0001645) (not measured)",1
2,"original value: ""nan"" -> HP: Sudden cardiac death (HP:0001645) (not measured)",11
3,"original value: ""Sudden death"" -> HP: Sudden cardiac death (HP:0001645) (observed)",11
4,"original value: ""No"" -> HP: Sudden cardiac death (HP:0001645) (excluded)",1


In [11]:
#auto = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr, delimiter=",")

In [12]:
comments_d = {
 'Arterial embolus': 'Thromboembolism',
 'Heart failure': 'Congestive heart failure',
 'Embolic stroke': 'Stroke',
 'Left atrial enlargement': 'Left atrial enlargement',
 'Renal failure': 'Renal insufficiency',
 'Heart transplant': 'Congestive heart failure'}
commentsMapper = OptionColumnMapper(column_name='COMMENTS',concept_recognizer=hpo_cr, option_d=comments_d)
column_mapper_list.append(commentsMapper)
commentsMapper.preview_column(df)

,mapping,count
0,Thromboembolism (HP:0001907) (observed),1
1,Congestive heart failure (HP:0001635) (observed),13
2,Stroke (HP:0001297) (observed),3
3,Left atrial enlargement (HP:0031295) (observed),1
4,Renal insufficiency (HP:0000083) (observed),1


In [13]:
conduction_d = {'Third-degree atrioventricular block': 'Third degree atrioventricular block',
 'permanent pacemaker': 'Abnormal atrioventricular conduction',
 'Sinus bradycardia': 'Sinus bradycardia',
 'Second-degree atrioventricular block': 'Second degree atrioventricular block',
 'first-degree atrioventricular block': 'First degree atrioventricular block',
 'First-degree atrioventricular block': 'First degree atrioventricular block',
 'Yes': 'Abnormal atrioventricular conduction',
 'Slow atrial fibrillation': 'Atrial fibrillation',
 'Yes; permanent pacemaker': 'Abnormal atrioventricular conduction'
}
excluded = {'No':'Abnormal atrioventricular conduction'}
conductionMapper = OptionColumnMapper(column_name='CONDUCTION -SYSTEM DISEASE',concept_recognizer=hpo_cr, option_d=conduction_d)
column_mapper_list.append(conductionMapper)
conductionMapper.preview_column(df)

,mapping,count
0,Third degree atrioventricular block (HP:0001709) (observed),5
1,Abnormal atrioventricular conduction (HP:0005150) (observed),25
2,Sinus bradycardia (HP:0001688) (observed),4
3,Second degree atrioventricular block (HP:0011706) (observed),4
4,First degree atrioventricular block (HP:0011705) (observed),5
5,Atrial fibrillation (HP:0005110) (observed),6


In [14]:
LMNA_transcript = "NM_005572.4"
vman = VariantManager(df=df, allele_1_column_name='NM_005572.4', individual_column_name="individual_id",
                      gene_symbol="LMNA", transcript=LMNA_transcript)

In [15]:
variant_d = vman.get_variant_d()
varMapper = VariantColumnMapper(variant_d=variant_d,
                                variant_column_name='NM_005572.4', 
                                default_genotype='heterozygous')

In [16]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="individual_id", 
                        age_of_onset_mapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
omim_id = "OMIM:115200"
omim_label = "Cardiomyopathy, dilated, 1A"
 	 
cmp1a = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=cmp1a)

In [17]:
individuals = encoder.get_individuals()
# We code the following terms as excluded (see top of notebook)
for i in individuals:
    progeroid = HpTerm(hpo_id="HP:0005328", label="Progeroid facial appearance", observed=False)
    lipodystrophy = HpTerm(hpo_id="HP:0009125", label="Lipodystrophy", observed=False)
    i.add_hpo_term(progeroid)
    i.add_hpo_term(lipodystrophy)

In [18]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,6
INFORMATION,NOT_MEASURED,30


In [19]:
individuals = cvalidator.get_error_free_individual_list()
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
A_III-1 (MALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Dilated cardiomyopathy (HP:0001644); Third degree atrioventricular block (HP:0001709); excluded: Atrial fibrillation (HP:0005110); excluded: Elevated circulating creatine kinase concentration (HP:0003236); excluded: Sudden cardiac death (HP:0001645); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
A_III-5 (FEMALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),excluded: Atrial fibrillation (HP:0005110); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Elevated circulating creatine kinase concentration (HP:0003236); excluded: Sudden cardiac death (HP:0001645); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
A_III-8 (FEMALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Atrial fibrillation (HP:0005110); Dilated cardiomyopathy (HP:0001644); Thromboembolism (HP:0001907); Sinus bradycardia (HP:0001688); Abnormal atrioventricular conduction (HP:0005150); excluded: Elevated circulating creatine kinase concentration (HP:0003236); excluded: Sudden cardiac death (HP:0001645); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
A_III-9 (MALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Dilated cardiomyopathy (HP:0001644); Congestive heart failure (HP:0001635); Third degree atrioventricular block (HP:0001709); excluded: Atrial fibrillation (HP:0005110); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
A_III-11 (FEMALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Atrial fibrillation (HP:0005110); Elevated circulating creatine kinase concentration (HP:0003236); Second degree atrioventricular block (HP:0011706); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Sudden cardiac death (HP:0001645); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
A_III-13 (MALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Atrial fibrillation (HP:0005110); Sinus bradycardia (HP:0001688); First degree atrioventricular block (HP:0011705); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Elevated circulating creatine kinase concentration (HP:0003236); excluded: Sudden cardiac death (HP:0001645); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
A_III-14 (FEMALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),First degree atrioventricular block (HP:0011705); excluded: Atrial fibrillation (HP:0005110); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Elevated circulating creatine kinase concentration (HP:0003236); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
A_III-15 (FEMALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Third degree atrioventricular block (HP:0001709); excluded: Atrial fibrillation (HP:0005110); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Sudden cardiac death (HP:0001645); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
B_III-1 (MALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.585C>A (heterozygous),Atrial fibrillation (HP:0005110); Dilated cardiomyopathy (HP:0001644); Sudden cardiac death (HP:0001645); Congestive heart failure (HP:0001635); Abnormal atrioventricular conduction (HP:0005150); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
B_III-3 (MALE; n/a),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.585C>A (heterozygous),Sudden cardiac death (HP:0001645); Dilated cardiomyopathy (HP:0001644)

In [20]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 39 GA4GH phenopackets to the directory phenopackets
